In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')


# ทำต่อจาก part 1 

In [3]:
def is_categorical(array_like):
    return array_like.dtype.name == 'category'

In [4]:
def agg_numeric(df, group_var, df_name) :
    for col in df:
        if col != group_var and 'SK_ID' in col : # เป็น SK_ID แต่ไม่ใช่่สิ่งที่ต้องการจะ group ก็ให้ลบทิ้ง col นั้นไปเลย
            df = df.drop(columns = col)
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number') # ทำตารางใหม่ เอาข้อมูลที่เป็น ตัวเลข เท่านั้น
    numeric_df[group_var] = group_ids # ใส่สิ่งที่จะ group ไปด้วย
    
    agg = numeric_df.groupby(group_var).agg(['count','mean','max','min','sum']).reset_index()
    
    columns = [group_var] # เปลี่ยนชื่อตาราง
    for var in agg.columns.levels[0]:
        if var != group_var:
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))
    agg.columns = columns
    
    # Remove the columns with all redundant values
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:,idx]
    
    return agg

In [5]:
def RenameColumn(df, group_var, df_name):
    # Need to create new column names
    columns = []

    # Iterate through the variables names
    for var in df.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in ['count', 'count_norm']:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    #  Rename the columns
    df.columns = columns

In [6]:
def remove_missing_columns(train, test, threshold = 90):
    # Calculate missing stats for train and test (remember to calculate a percent!)
    train_miss = pd.DataFrame(train.isnull().sum())
    train_miss['percent'] = 100 * train_miss[0] / len(train)
    
    test_miss = pd.DataFrame(test.isnull().sum())
    test_miss['percent'] = 100 * test_miss[0] / len(test)
    
    # list of missing columns for train and test
    missing_train_columns = list(train_miss.index[train_miss['percent'] > threshold])
    missing_test_columns = list(test_miss.index[test_miss['percent'] > threshold])
    
    # Combine the two lists together
    missing_columns = list(set(missing_train_columns + missing_test_columns))
    
    # Print information
    print('There are %d columns with greater than %d%% missing values.' % (len(missing_columns), threshold))
    
    # Drop the missing columns and return
    train = train.drop(columns = missing_columns)
    test = test.drop(columns = missing_columns)
    
    return train, test

# Function to Convert Data Types

This will help reduce memory usage by using more efficient types for the variables. For example `category` is often a better type than `object` (unless the number of unique categories is close to the number of rows in the dataframe).

In [7]:
import sys

def return_size(df):
    """Return size of dataframe in gigabytes"""
    return round(sys.getsizeof(df) / 1e9, 2)

def convert_types(df, print_info = False):
    
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print('Original Memory Usage:',round(original_memory / 1e9, 2), 'gb.')
        print('New Memory Usage:,',round(new_memory / 1e9, 2),'gb.')
        
    return df

In [8]:
import gc

Load data จาก part1 มา

In [11]:
app_test = pd.read_csv('./test_part1.csv')
app_train = pd.read_csv('./train_part1.csv')

In [12]:
app_train = convert_types(app_train, print_info=True)
app_train.head()

Original Memory Usage: 0.41 gb.
New Memory Usage:, 0.2 gb.


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_ANNUITY,AMT_GOODS_PRICE,...,client_bureau_balance_STATUS_0_count_min,client_bureau_balance_STATUS_X_count_norm_sum,client_bureau_balance_MONTHS_BALANCE_count_min,client_bureau_balance_STATUS_0_count_norm_sum,client_bureau_balance_STATUS_X_count_mean,client_bureau_balance_STATUS_0_count_mean,client_bureau_balance_STATUS_0_count_max,client_bureau_balance_STATUS_0_count_sum,client_bureau_balance_STATUS_C_count_max,client_bureau_balance_STATUS_C_count_sum
0,100002,True,0.083459,0.101419,0.085002,0.079616,0,202500.0,24700.5,351000.0,...,2.0,1.295455,4.0,3.255682,1.875,5.625,18.0,45.0,13.0,23.0
1,100003,False,0.083459,0.069993,0.085002,0.083249,0,270000.0,35698.5,1129500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,False,0.054783,0.101419,0.072437,0.079616,0,67500.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,False,0.083459,0.069993,0.085002,0.079616,0,135000.0,29686.5,297000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,False,0.083459,0.101419,0.085002,0.079616,0,121500.0,21865.5,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
app_test = convert_types(app_test, print_info=True)
app_test.head()

Original Memory Usage: 0.07 gb.
New Memory Usage:, 0.03 gb.


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,client_bureau_balance_STATUS_0_count_min,client_bureau_balance_STATUS_X_count_norm_sum,client_bureau_balance_MONTHS_BALANCE_count_min,client_bureau_balance_STATUS_0_count_norm_sum,client_bureau_balance_STATUS_X_count_mean,client_bureau_balance_STATUS_0_count_mean,client_bureau_balance_STATUS_0_count_max,client_bureau_balance_STATUS_0_count_sum,client_bureau_balance_STATUS_C_count_max,client_bureau_balance_STATUS_C_count_sum
0,100001,0.083459,0.069993,0.085002,0.079616,0,135000.0,20560.5,450000.0,0.08183,...,1.0,1.502129,2.0,2.356557,4.285714,4.428571,12.0,31.0,44.0,110.0
1,100005,0.083459,0.101419,0.085002,0.079616,0,99000.0,17370.0,180000.0,0.08183,...,2.0,0.410256,3.0,2.205128,0.666667,4.666667,7.0,14.0,5.0,5.0
2,100013,0.083459,0.101419,0.072437,0.079616,0,202500.0,69777.0,630000.0,NaN,...,0.0,1.018182,40.0,1.282872,10.250000,19.750000,34.0,79.0,44.0,103.0
3,100028,0.083459,0.069993,0.085002,0.079616,2,315000.0,49018.5,1575000.0,0.08183,...,0.0,3.125213,9.0,4.527848,11.083333,13.666667,34.0,164.0,62.0,263.0
4,100038,0.083459,0.101419,0.072437,0.083249,1,180000.0,32067.0,625500.0,0.08183,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
app_train.dtypes.value_counts()

float32    130
int32       32
bool         6
dtype: int64

### Previous data

In [16]:
previous = pd.read_csv('../input/previous_application.csv')
previous = convert_types(previous, print_info=True)
previous.head()

Original Memory Usage: 0.49 gb.
New Memory Usage:, 0.16 gb.


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430054,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615234,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735352,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335938,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.394531,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


คำนวณข้อมูลตัวเลขของตาราง previous

In [17]:
previous_agg = agg_numeric(previous, 'SK_ID_CURR', 'previous')
print('Previous aggregation shape:', previous_agg.shape)
previous_agg.head()

Previous aggregation shape: (338857, 81)


,previous_DAYS_DECISION_sum,previous_DAYS_DECISION_min,previous_DAYS_DECISION_mean,previous_DAYS_DECISION_max,previous_DAYS_FIRST_DUE_sum,previous_DAYS_FIRST_DUE_min,previous_DAYS_FIRST_DUE_mean,previous_DAYS_FIRST_DUE_max,previous_DAYS_LAST_DUE_sum,previous_DAYS_LAST_DUE_min,...,previous_DAYS_FIRST_DRAWING_min,previous_DAYS_FIRST_DRAWING_mean,previous_DAYS_FIRST_DRAWING_max,previous_DAYS_FIRST_DRAWING_sum,previous_RATE_INTEREST_PRIMARY_min,previous_RATE_INTEREST_PRIMARY_mean,previous_RATE_INTEREST_PRIMARY_max,previous_RATE_INTEREST_PRIVILEGED_min,previous_RATE_INTEREST_PRIVILEGED_mean,previous_RATE_INTEREST_PRIVILEGED_max
0,-1740,-1740,-1740.0,-1740,-1709.0,-1709.0,-1709.000000,-1709.0,-1619.0,-1619.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
1,-606,-606,-606.0,-606,-565.0,-565.0,-565.000000,-565.0,-25.0,-25.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
2,-3915,-2341,-1305.0,-746,-3823.0,-2310.0,-1274.333374,-716.0,-3163.0,-1980.0,...,365243.0,365243.0,365243.0,1095729.0,NaN,NaN,NaN,NaN,NaN,NaN
3,-815,-815,-815.0,-815,-784.0,-784.0,-784.000000,-784.0,-724.0,-724.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
4,-1072,-757,-536.0,-315,-706.0,-706.0,-706.000000,-706.0,-466.0,-466.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN


### function สำหรับการทำ Target Encoding

In [9]:
def count_categorical3(app_train, app_test, cash, group_var, df_name) :
    app_train_cols = [group_var,'TARGET']
    app_train_temp = app_train[app_train_cols]
    
    app_test_cols = [group_var]
    app_test_temp = app_test[app_test_cols]
    
    skIdCurr = cash[group_var]
    cash_obj_cols = cash.select_dtypes('category')
    cash_obj_cols[group_var] = skIdCurr
    
    cash_train = app_train_temp.merge(cash_obj_cols, how='left')
    cash_test = app_test_temp.merge(cash_obj_cols, how='left')
    
    TargetEncoder3(cash_train, cash_test)
    
    app_train_cash_group = cash_train.drop(columns=["TARGET"]).groupby(group_var).agg(['sum', 'mean'])
    app_test_cash_group = cash_test.groupby(group_var).agg(['sum', 'mean'])
    
    RenameColumn(app_train_cash_group,group_var, df_name)
    RenameColumn(app_test_cash_group,group_var, df_name)
    
    app_train_cash_group = app_train_temp.merge(app_train_cash_group, left_on = group_var, right_index = True, how="left")
    app_test_cash_group = app_test_temp.merge(app_test_cash_group, left_on = group_var, right_index = True, how="left")
    import gc
    gc.enable()
    del app_train_temp, app_test_temp, cash_obj_cols, cash_train, cash_test, skIdCurr
    gc.collect()
    return app_train_cash_group, app_test_cash_group

In [10]:
def TargetEncoder3(app_train,app_test) :
    for col in app_train :
        if is_categorical(app_train[col]):
            print(col)
            cumsum = app_train.groupby(col)['TARGET'].sum()
            cumcnt = app_train.groupby(col)['TARGET'].count()
            temp_train = app_train[col]
            temp_train = temp_train.cat.codes
            temp_test = app_test[col]
            temp_test = temp_test.cat.codes
            
            app_tr = app_train[col]
            app_te = app_test[col]
            
            print(list(cumsum.index))
            for x in list(cumsum.index) :
                c = cumsum.loc[x]/cumcnt.loc[x]
                temp_train.loc[app_train[col] == x] = c
                app_tr = temp_train
                temp_test.loc[app_test[col] == x] = c
                app_te = temp_test
            app_train[col] = app_tr.fillna(app_tr.mean())
            app_test[col] = app_te.convert_objects(convert_numeric=True).fillna(app_train[col].mean())
            gc.enable()
            del temp_train, temp_test
            gc.collect()

In [20]:
previous.dtypes.value_counts()

float32     15
int32        5
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
bool         1
category     1
category     1
category     1
dtype: int64

In [21]:
previous_counts_train, previous_counts_test = count_categorical3(app_train, app_test, previous, group_var='SK_ID_CURR', df_name='previous')

NAME_CONTRACT_TYPE
['Cash loans', 'Consumer loans', 'Revolving loans', 'XNA']
WEEKDAY_APPR_PROCESS_START
['FRIDAY', 'MONDAY', 'SATURDAY', 'SUNDAY', 'THURSDAY', 'TUESDAY', 'WEDNESDAY']
FLAG_LAST_APPL_PER_CONTRACT
['N', 'Y']
NAME_CASH_LOAN_PURPOSE
['Building a house or an annex', 'Business development', 'Buying a garage', 'Buying a holiday home / land', 'Buying a home', 'Buying a new car', 'Buying a used car', 'Car repairs', 'Education', 'Everyday expenses', 'Furniture', 'Gasification / water supply', 'Hobby', 'Journey', 'Medicine', 'Money for a third person', 'Other', 'Payments on other loans', 'Purchase of electronic equipment', 'Refusal to name the goal', 'Repairs', 'Urgent needs', 'Wedding / gift / holiday', 'XAP', 'XNA']
NAME_CONTRACT_STATUS
['Approved', 'Canceled', 'Refused', 'Unused offer']
NAME_PAYMENT_TYPE
['Cash through the bank', 'Cashless from the account of the employer', 'Non-cash from your account', 'XNA']
CODE_REJECT_REASON
['CLIENT', 'HC', 'LIMIT', 'SCO', 'SCOFR', 'SYSTE

In [22]:
previous_counts_train.dtypes.value_counts()

float64    32
int32       1
bool        1
dtype: int64

In [23]:
previous_counts_train.head()

,SK_ID_CURR,TARGET,previous_NAME_CONTRACT_TYPE_count,previous_NAME_CONTRACT_TYPE_count_norm,previous_WEEKDAY_APPR_PROCESS_START_count,previous_WEEKDAY_APPR_PROCESS_START_count_norm,previous_FLAG_LAST_APPL_PER_CONTRACT_count,previous_FLAG_LAST_APPL_PER_CONTRACT_count_norm,previous_NAME_CASH_LOAN_PURPOSE_count,previous_NAME_CASH_LOAN_PURPOSE_count_norm,...,previous_NAME_PRODUCT_TYPE_count,previous_NAME_PRODUCT_TYPE_count_norm,previous_CHANNEL_TYPE_count,previous_CHANNEL_TYPE_count_norm,previous_NAME_SELLER_INDUSTRY_count,previous_NAME_SELLER_INDUSTRY_count_norm,previous_NAME_YIELD_GROUP_count,previous_NAME_YIELD_GROUP_count_norm,previous_PRODUCT_COMBINATION_count,previous_PRODUCT_COMBINATION_count_norm
0,100002,True,0.077100,0.077100,0.083469,0.083469,0.086441,0.086441,0.082806,0.082806,...,0.083548,0.083548,0.073697,0.073697,0.103676,0.103676,0.071321,0.071321,0.081343,0.081343
1,100003,False,0.245457,0.081819,0.251521,0.083840,0.259322,0.086441,0.252785,0.084262,...,0.247975,0.082658,0.246367,0.082122,0.230254,0.076751,0.232185,0.077395,0.206202,0.068734
2,100004,False,0.077100,0.077100,0.087504,0.087504,0.086441,0.086441,0.082806,0.082806,...,0.083548,0.083548,0.076055,0.076055,0.092070,0.092070,0.080432,0.080432,0.078834,0.078834
3,100006,False,0.819860,0.091096,0.785081,0.087231,0.777966,0.086441,0.767091,0.085232,...,0.741255,0.082362,0.787029,0.087448,0.796163,0.088463,0.808508,0.089834,0.783740,0.087082
4,100007,False,0.519230,0.086538,0.509575,0.084929,0.518644,0.086441,0.514306,0.085718,...,0.534422,0.089070,0.542110,0.090352,0.505782,0.084297,0.527650,0.087942,0.514170,0.085695


In [24]:
#previous_counts_test

### รวมตารางกลับไปที่ app_train และ app_test

In [25]:
app_train = app_train.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')
app_train = app_train.merge(previous_counts_train.drop(columns='TARGET'), on ='SK_ID_CURR', how = 'left')

app_test = app_test.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(previous_counts_test, on ='SK_ID_CURR', how = 'left')



### ลบ missing column > 90%

In [27]:
app_train, app_test = remove_missing_columns(app_train, app_test)
app_train.to_csv('train_part2_step1_fix.csv', index = False)
app_test.to_csv('test_part2_step1_fix.csv', index = False)

There are 6 columns with greater than 90% missing values.


In [28]:
app_train.shape

(307511, 274)

#### Remove variables to free memory

In [29]:
import gc
gc.enable()
del previous, previous_agg, previous_counts_train, previous_counts_test, app_test, app_train
gc.collect()

70

# Applying to More Data

### Function to Aggregate Stats at the Client Level

In [18]:
def aggregate_client(df, group_vars, df_names):
    """Aggregate a dataframe with data at the loan level 
    at the client level
    
    Args:
        df (dataframe): data at the loan level
        group_vars (list of two strings): grouping variables for the loan 
        and then the client (example ['SK_ID_PREV', 'SK_ID_CURR'])
        names (list of two strings): names to call the resulting columns
        (example ['cash', 'client'])
        
    Returns:
        df_client (dataframe): aggregated numeric stats at the client level. 
        Each client will have a single row with all the numeric data aggregated
    """
    import gc
    # Aggregate the numeric columns
    df_agg = agg_numeric(df, group_var = group_vars[0], df_name = df_names[0])
    
    # If there are categorical variables
    if not any(df.dtypes == 'categorical'):
        # Merge to get the client id in dataframe
        df_by_loan = df_agg.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')
        
        gc.enable()
        del df_agg
        gc.collect()
        
        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])
        
        # Aggregate numeric stats by column
        df_by_client = agg_numeric(df_by_loan, group_var = group_vars[1], df_name = df_names[1])
        
    # Memory management
    gc.enable()
    del df, df_by_loan
    gc.collect()

    return df_by_client

## Monthly Cash Data

In [31]:
cash = pd.read_csv('../input/POS_CASH_balance.csv')
cash = convert_types(cash, print_info=True)
cash.head()

Original Memory Usage: 0.64 gb.
New Memory Usage:, 0.29 gb.


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [34]:
cash_by_client = aggregate_client(cash, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['cash', 'client'])
cash_by_client.head()

,client_cash_MONTHS_BALANCE_sum_sum,client_cash_MONTHS_BALANCE_min_sum,client_cash_MONTHS_BALANCE_mean_sum,client_cash_MONTHS_BALANCE_max_sum,client_cash_MONTHS_BALANCE_sum_min,client_cash_MONTHS_BALANCE_sum_mean,client_cash_MONTHS_BALANCE_sum_max,client_cash_MONTHS_BALANCE_min_min,client_cash_MONTHS_BALANCE_mean_min,client_cash_MONTHS_BALANCE_max_min,...,client_cash_CNT_INSTALMENT_FUTURE_max_sum,client_cash_CNT_INSTALMENT_min_sum,client_cash_CNT_INSTALMENT_mean_sum,client_cash_CNT_INSTALMENT_max_sum,client_cash_CNT_INSTALMENT_count_sum,client_cash_CNT_INSTALMENT_FUTURE_count_sum,client_cash_MONTHS_BALANCE_count_sum,client_cash_CNT_INSTALMENT_FUTURE_sum_sum,client_cash_CNT_INSTALMENT_sum_sum,SK_ID_CURR
0,-2887,-669,-653.0,-637,-378,-320.777778,-275,-96,-94.5,-93,...,28.0,36.0,36.000000,36.0,41,41,41,62.0,164.0,100001
1,-3610,-361,-190.0,-19,-190,-190.000000,-190,-19,-10.0,-1,...,456.0,456.0,456.000000,456.0,361,361,361,5415.0,8664.0,100002
2,-13240,-1348,-1226.0,-1104,-858,-472.857143,-172,-77,-71.5,-66,...,288.0,248.0,283.000000,288.0,272,272,272,1608.0,2840.0,100003
3,-408,-108,-102.0,-96,-102,-102.000000,-102,-27,-25.5,-24,...,16.0,12.0,15.000000,16.0,16,16,16,36.0,60.0,100004
4,-2420,-275,-220.0,-165,-220,-220.000000,-220,-25,-20.0,-15,...,132.0,99.0,128.699997,132.0,110,110,121,792.0,1287.0,100005


In [35]:
app_train = pd.read_csv('./train_part2_step1_fix.csv')
app_test = pd.read_csv('./test_part2_step1_fix.csv')

In [36]:
cash_train_obj, cash_test_obj = count_categorical3(app_train, app_test, cash, 'SK_ID_CURR', 'cash')

NAME_CONTRACT_STATUS
['Active', 'Amortized debt', 'Approved', 'Canceled', 'Completed', 'Demand', 'Returned to the store', 'Signed', 'XNA']


In [37]:
print('Cash by Client Shape: ', cash_by_client.shape)
app_train = app_train.merge(cash_by_client, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(cash_by_client, on = 'SK_ID_CURR', how = 'left')
app_train = app_train.merge(cash_train_obj.drop(columns='TARGET'), on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(cash_test_obj, on = 'SK_ID_CURR', how = 'left')

Cash by Client Shape:  (337252, 95)


In [38]:
gc.enable()
del cash, cash_by_client
gc.collect()

70

In [39]:
app_train, app_test = remove_missing_columns(app_train, app_test)

app_train.to_csv('train_part2_step2.csv', index = False)
app_test.to_csv('test_part2_step2.csv', index = False)

There are 0 columns with greater than 90% missing values.


In [40]:
gc.enable()
del app_train, app_test
gc.collect()

42

## Monthly Credit Data

In [42]:
credit = pd.read_csv('../input/credit_card_balance.csv')
credit = convert_types(credit, print_info = True)
credit.head()

Original Memory Usage: 0.71 gb.
New Memory Usage:, 0.34 gb.


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [43]:
credit_by_client = aggregate_client(credit, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['credit', 'client'])
credit_by_client.head()

,client_credit_MONTHS_BALANCE_sum_sum,client_credit_MONTHS_BALANCE_min_sum,client_credit_MONTHS_BALANCE_mean_sum,client_credit_MONTHS_BALANCE_sum_min,client_credit_MONTHS_BALANCE_sum_mean,client_credit_MONTHS_BALANCE_sum_max,client_credit_MONTHS_BALANCE_max_sum,client_credit_MONTHS_BALANCE_min_min,client_credit_MONTHS_BALANCE_min_mean,client_credit_MONTHS_BALANCE_min_max,...,client_credit_AMT_DRAWINGS_ATM_CURRENT_mean_max,client_credit_AMT_PAYMENT_CURRENT_mean_min,client_credit_AMT_PAYMENT_CURRENT_mean_mean,client_credit_AMT_PAYMENT_CURRENT_mean_max,client_credit_AMT_PAYMENT_CURRENT_max_min,client_credit_AMT_PAYMENT_CURRENT_max_mean,client_credit_AMT_PAYMENT_CURRENT_max_max,client_credit_AMT_DRAWINGS_ATM_CURRENT_max_min,client_credit_AMT_DRAWINGS_ATM_CURRENT_max_mean,client_credit_AMT_DRAWINGS_ATM_CURRENT_max_max
0,-126,-36,-21.0,-21,-21.0,-21,-6,-6,-6.0,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-210826,-5550,-2849.0,-2849,-2849.0,-2849,-148,-75,-75.0,-75,...,2432.432373,4843.063965,4843.063965,4843.063965,55485.0,55485.0,55485.0,180000.0,180000.0,180000.0
2,-446976,-9216,-4656.0,-4656,-4656.0,-4656,-96,-96,-96.0,-96,...,6350.000000,7168.346191,7168.346191,7168.346191,153675.0,153675.0,153675.0,157500.0,157500.0,157500.0
3,-2890,-306,-170.0,-170,-170.0,-170,-34,-18,-18.0,-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-480,-88,-60.0,-60,-60.0,-60,-32,-11,-11.0,-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
app_train = pd.read_csv('./train_part2_step2.csv')
app_test = pd.read_csv('./test_part2_step2.csv')

In [46]:
credit_train_obj, credit_test_obj = count_categorical3(app_train, app_test, credit, 'SK_ID_CURR', 'credit')

NAME_CONTRACT_STATUS
['Active', 'Approved', 'Completed', 'Demand', 'Refused', 'Sent proposal', 'Signed']


In [47]:
print('Credit by client shape: ', credit_by_client.shape)

app_train = app_train.merge(credit_by_client, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(credit_by_client, on = 'SK_ID_CURR', how = 'left')
app_train = app_train.merge(credit_train_obj.drop(columns='TARGET'), on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(credit_test_obj, on = 'SK_ID_CURR', how = 'left')

gc.enable()
del credit, credit_by_client, credit_test_obj, credit_train_obj
gc.collect()

Credit by client shape:  (103558, 332)


56

In [48]:
app_train, app_test = remove_missing_columns(app_train, app_test)
app_train.to_csv('train_part2_step3.csv', index = False)
app_test.to_csv('test_part2_step3.csv', index = False)

There are 0 columns with greater than 90% missing values.


In [49]:
gc.enable()
del app_train, app_test
gc.collect()

42

### Installment Payments

In [19]:
installments = pd.read_csv('../input/installments_payments.csv')
installments = convert_types(installments, print_info = True)
installments.head()

Original Memory Usage: 0.87 gb.
New Memory Usage:, 0.44 gb.


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525024,1716.525024
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000000,25425.000000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130859,24350.130859
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040039,2160.584961


In [20]:
installments_by_client = aggregate_client(installments, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['installments', 'client'])
installments_by_client.head()

,client_installments_DAYS_ENTRY_PAYMENT_sum_sum,client_installments_DAYS_INSTALMENT_sum_sum,client_installments_DAYS_ENTRY_PAYMENT_min_sum,client_installments_DAYS_INSTALMENT_min_sum,client_installments_DAYS_ENTRY_PAYMENT_mean_sum,client_installments_DAYS_INSTALMENT_mean_sum,client_installments_DAYS_ENTRY_PAYMENT_max_sum,client_installments_DAYS_INSTALMENT_max_sum,client_installments_DAYS_INSTALMENT_sum_min,client_installments_DAYS_ENTRY_PAYMENT_sum_min,...,client_installments_AMT_INSTALMENT_min_sum,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_INSTALMENT_sum_max,client_installments_AMT_PAYMENT_mean_sum,client_installments_AMT_INSTALMENT_mean_sum,client_installments_AMT_INSTALMENT_max_sum,client_installments_AMT_PAYMENT_max_sum,SK_ID_CURR,client_installments_AMT_PAYMENT_sum_sum,client_installments_AMT_INSTALMENT_sum_sum
0,-52813.0,-52598.0,-15608.0,-15584.0,-15365.0,-15314.0,-15080.0,-15044.0,-8658.0,-8647.0,...,2.774678e+04,2.925090e+04,2.925090e+04,4.119593e+04,4.119593e+04,8.153775e+04,8.153775e+04,100001,1.528387e+05,1.528387e+05
1,-113867.0,-106495.0,-11153.0,-10735.0,-5993.0,-5605.0,-931.0,-475.0,-5605.0,-5993.0,...,1.757837e+05,2.196257e+05,2.196257e+05,2.196257e+05,2.196257e+05,1.008781e+06,1.008781e+06,100002,4.172888e+06,4.172888e+06
2,-367137.0,-365546.0,-37757.0,-37514.0,-34633.0,-34454.0,-31594.0,-31394.0,-25740.0,-25821.0,...,1.154108e+06,1.150977e+06,1.150977e+06,1.618865e+06,1.618865e+06,4.394102e+06,4.394102e+06,100003,1.134881e+07,1.134881e+07
3,-6855.0,-6786.0,-2385.0,-2352.0,-2285.0,-2262.0,-2181.0,-2172.0,-2262.0,-2285.0,...,1.607175e+04,2.128846e+04,2.128846e+04,2.128846e+04,2.128846e+04,3.172189e+04,3.172189e+04,100004,6.386539e+04,6.386539e+04
4,-49374.0,-47466.0,-6624.0,-6354.0,-5486.0,-5274.0,-4230.0,-4194.0,-5274.0,-5486.0,...,4.331880e+04,5.616184e+04,5.616184e+04,5.616184e+04,5.616184e+04,1.589062e+05,1.589062e+05,100005,5.054566e+05,5.054566e+05


In [21]:
app_train = pd.read_csv('./train_part2_step3.csv')
app_test = pd.read_csv('./test_part2_step3.csv')

In [22]:
print('Installments by client shape: ', installments_by_client.shape)

app_train = app_train.merge(installments_by_client, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(installments_by_client, on = 'SK_ID_CURR', how = 'left')

gc.enable()
del installments, installments_by_client
gc.collect()

Installments by client shape:  (339587, 107)


21

In [23]:
app_train, app_test = remove_missing_columns(app_train, app_test)

There are 0 columns with greater than 90% missing values.


In [24]:
print('Final Training Shape: ', app_train.shape)
print('Final Testing Shape: ', app_test.shape)

Final Training Shape:  (307511, 809)
Final Testing Shape:  (48744, 808)


In [25]:
print('Final training size:', return_size(app_train))
print('Final testing size:', return_size(app_test))

Final training size: 1.88
Final testing size: 0.3


In [28]:
app_train.to_csv('train_part2_step4.csv', index = False)
app_test.to_csv('test_part2_step4.csv', index = False)

In [11]:
# Function to calculate correlations with the target for a dataframe
def target_corrs(df):

    # List of correlations
    corrs = []

    # Iterate through the columns 
    for col in df.columns:
        #print(col)
        # Skip the target column
        if col != 'TARGET':
            # Calculate correlation with the target
            corr = df['TARGET'].corr(df[col])

            # Append the list as a tuple
            corrs.append((col, corr))
            
    # Sort by absolute magnitude of correlations
    corrs = sorted(corrs, key = lambda x: abs(x[1]), reverse = True)
    
    return corrs

In [29]:
app_train = pd.read_csv('./train_part2_step4.csv')
app_test = pd.read_csv('./test_part2_step4.csv')

## Correlations

First let's look at the correlations of the variables with the target. We can see in any of the variables we created have a greater correlation than those already present in the training data (from `application`). 

In [30]:
# Calculate all correlations in dataframe
corrs = app_train.corr()

In [31]:
corrs = corrs.sort_values('TARGET', ascending = False)

# Ten most positive correlations
pd.DataFrame(corrs['TARGET'].head(10))

,TARGET
TARGET,1.000000
client_credit_CNT_DRAWINGS_ATM_CURRENT_mean_max,0.108340
client_credit_CNT_DRAWINGS_ATM_CURRENT_mean_mean,0.107713
client_credit_CNT_DRAWINGS_ATM_CURRENT_mean_min,0.107636
client_credit_CNT_DRAWINGS_CURRENT_max_max,0.101389
client_credit_CNT_DRAWINGS_CURRENT_max_mean,0.100921
client_credit_CNT_DRAWINGS_CURRENT_max_min,0.100648
client_bureau_balance_MONTHS_BALANCE_min_mean,0.089038
client_credit_AMT_BALANCE_mean_max,0.087617
client_credit_AMT_BALANCE_mean_mean,0.087177


In [32]:
# Ten most negative correlations
pd.DataFrame(corrs['TARGET'].dropna().tail(10))

,TARGET
client_installments_AMT_PAYMENT_min_sum,-0.058401
client_credit_MONTHS_BALANCE_count_sum,-0.058875
client_credit_MONTHS_BALANCE_sum_count,-0.060481
client_credit_MONTHS_BALANCE_count_max,-0.060807
client_credit_MONTHS_BALANCE_count_mean,-0.060987
client_bureau_balance_STATUS_C_count_max,-0.061083
client_credit_MONTHS_BALANCE_count_min,-0.061265
EXT_SOURCE_1,-0.155317
EXT_SOURCE_2,-0.160472
EXT_SOURCE_3,-0.178919


### Collinear Variables

We can calculate not only the correlations of the variables with the target, but also the correlation of each variable with every other variable. This will allow us to see if there are highly collinear variables that should perhaps be removed from the data. 

Let's look for any variables that have a greather than 0.8 correlation with other variables.

In [33]:
# Set the threshold
threshold = 0.8

# Empty dictionary to hold correlated variables
above_threshold_vars = {}

# For each column, record the variables that are above the threshold
for col in corrs:
    above_threshold_vars[col] = list(corrs.index[corrs[col] > threshold])

For each of these pairs of highly correlated variables, we only want to remove one of the variables. The following code creates a set of variables to remove by only adding one of each pair. 

In [34]:
# Track columns to remove and columns already examined
cols_to_remove = []
cols_seen = []
cols_to_remove_pair = []

# Iterate through columns and correlated columns
for key, value in above_threshold_vars.items():
    # Keep track of columns already examined
    cols_seen.append(key)
    for x in value:
        if x == key:
            next
        else:
            # Only want to remove one in a pair
            if x not in cols_seen:
                cols_to_remove.append(x)
                cols_to_remove_pair.append(key)
            
cols_to_remove = list(set(cols_to_remove))
print('Number of columns to remove: ', len(cols_to_remove))

Number of columns to remove:  506


We can remove these columns from both the training and the testing datasets. We will have to compare performance after removing these variables with performance keeping these variables (the raw csv files we saved earlier).

In [35]:
app_train = app_train.drop(columns = cols_to_remove)
app_test = app_test.drop(columns = cols_to_remove)

In [36]:
print('Training Corrs Removed Shape: ', app_train.shape)
print('Testing Corrs Removed Shape: ', app_train.shape)

Training Corrs Removed Shape:  (307511, 303)
Testing Corrs Removed Shape:  (307511, 303)


In [37]:
app_train.to_csv('train_part2_complete.csv', index = False)
app_test.to_csv('test_part2_complete.csv', index = False)

In [12]:
app_train = pd.read_csv('./train_part2_complete.csv')
app_test = pd.read_csv('./test_part2_complete.csv')

In [13]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_ANNUITY,AMT_GOODS_PRICE,...,client_installments_AMT_INSTALMENT_min_max,client_installments_AMT_INSTALMENT_mean_min,client_installments_AMT_INSTALMENT_max_min,client_installments_AMT_INSTALMENT_mean_mean,client_installments_AMT_INSTALMENT_max_mean,client_installments_AMT_INSTALMENT_sum_min,client_installments_AMT_PAYMENT_max_max,client_installments_AMT_PAYMENT_min_sum,client_installments_AMT_PAYMENT_mean_sum,client_installments_AMT_PAYMENT_sum_sum
0,100002,True,0.083459,0.101419,0.085002,0.079616,0,202500.0,24700.5,351000.0,...,9251.775,11559.2470,53093.746,11559.247,53093.746,219625.700,53093.746,175783.73,219625.690,4.172888e+06
1,100003,False,0.083459,0.069993,0.085002,0.083249,0,270000.0,35698.5,1129500.0,...,98356.990,6731.1150,6737.310,64754.586,175764.050,80773.380,560835.400,1154108.20,1618864.600,1.134881e+07
2,100004,False,0.054783,0.101419,0.072437,0.079616,0,67500.0,6750.0,135000.0,...,5357.250,7096.1550,10573.965,7096.155,10573.965,21288.465,10573.965,16071.75,21288.465,6.386539e+04
3,100006,False,0.083459,0.069993,0.085002,0.079616,0,135000.0,29686.5,297000.0,...,691786.900,5018.2646,15159.645,62947.086,66116.266,25091.324,691786.900,994476.70,1007153.400,3.719995e+06
4,100007,False,0.083459,0.101419,0.085002,0.079616,0,121500.0,21865.5,513000.0,...,22485.690,1833.0391,1834.290,12666.444,12677.324,18330.390,22678.785,483756.38,806127.940,1.139621e+07


### Exterior Sources

The 3 variables with the strongest negative correlations with the target are `EXT_SOURCE_1`, `EXT_SOURCE_2`, and `EXT_SOURCE_3`.
According to the documentation, these features represent a "normalized score from external data source". I'm not sure what this exactly means, but it may be a cumulative sort of credit rating made using numerous sources of data. 

Let's take a look at these variables.

First, we can show the correlations of the `EXT_SOURCE` features with the target and with each other.

In [14]:
# Calculate all correlations in dataframe
corrs = app_train.corr()

In [15]:
corrs = corrs.sort_values('TARGET', ascending = False)

# Ten most positive correlations
pd.DataFrame(corrs['TARGET'].head(10))

,TARGET
TARGET,1.000000
client_credit_CNT_DRAWINGS_ATM_CURRENT_mean_mean,0.107713
client_bureau_balance_MONTHS_BALANCE_min_mean,0.089038
OCCUPATION_TYPE,0.081516
bureau_CREDIT_ACTIVE_count_norm,0.079848
DAYS_BIRTH,0.078239
client_credit_AMT_INST_MIN_REGULARITY_mean_mean,0.073720
ORGANIZATION_TYPE,0.072340
bureau_DAYS_CREDIT_UPDATE_mean,0.068927
client_credit_CNT_DRAWINGS_POS_CURRENT_max_mean,0.068655


In [16]:
pd.DataFrame(corrs['TARGET'].tail(10))

,TARGET
previous_DAYS_FIRST_DRAWING_mean,-0.048803
client_bureau_balance_STATUS_C_count_sum,-0.048912
client_bureau_balance_STATUS_C_count_norm_mean,-0.055936
client_installments_AMT_PAYMENT_min_sum,-0.058401
client_bureau_balance_STATUS_C_count_max,-0.061083
EXT_SOURCE_1,-0.155317
EXT_SOURCE_2,-0.160472
EXT_SOURCE_3,-0.178919
client_credit_SK_DPD_min_mean,NaN
client_credit_SK_DPD_min_sum,NaN


In [17]:
# Extract the EXT_SOURCE variables and show correlations
cols_ext_data = ['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3','OCCUPATION_TYPE', 'client_bureau_balance_MONTHS_BALANCE_min_mean','DAYS_BIRTH','bureau_CREDIT_ACTIVE_count_norm']
cols_ext_data_test = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3','OCCUPATION_TYPE', 'client_bureau_balance_MONTHS_BALANCE_min_mean','DAYS_BIRTH','bureau_CREDIT_ACTIVE_count_norm']
ext_data = app_train[cols_ext_data]
ext_data_corrs = ext_data.corr()
ext_data_corrs

,TARGET,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OCCUPATION_TYPE,client_bureau_balance_MONTHS_BALANCE_min_mean,DAYS_BIRTH,bureau_CREDIT_ACTIVE_count_norm
TARGET,1.000000,-0.155317,-0.160472,-0.178919,0.081516,0.089038,0.078239,0.079848
EXT_SOURCE_1,-0.155317,1.000000,0.213982,0.186846,-0.266111,-0.195863,-0.600610,-0.094333
EXT_SOURCE_2,-0.160472,0.213982,1.000000,0.109167,-0.123320,-0.109528,-0.091996,-0.042502
EXT_SOURCE_3,-0.178919,0.186846,0.109167,1.000000,-0.002661,-0.400537,-0.205478,-0.361478
OCCUPATION_TYPE,0.081516,-0.266111,-0.123320,-0.002661,1.000000,0.040810,-0.008991,-0.005828
client_bureau_balance_MONTHS_BALANCE_min_mean,0.089038,-0.195863,-0.109528,-0.400537,0.040810,1.000000,0.200788,0.483877
DAYS_BIRTH,0.078239,-0.600610,-0.091996,-0.205478,-0.008991,0.200788,1.000000,0.115428
bureau_CREDIT_ACTIVE_count_norm,0.079848,-0.094333,-0.042502,-0.361478,-0.005828,0.483877,0.115428,1.000000


In [18]:
# Make a new dataframe for polynomial features
poly_features = app_train[cols_ext_data]
poly_features_test = app_test[cols_ext_data_test]

# imputer for handling missing values
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')

poly_target = poly_features['TARGET']

poly_features = poly_features.drop(columns = ['TARGET'])

# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)

from sklearn.preprocessing import PolynomialFeatures
                                  
# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)

/home/burin/.env/lib/python3.5/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [19]:
# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

Polynomial Features shape:  (307511, 120)


In [20]:
poly_transformer.get_feature_names(input_features = cols_ext_data_test)[:35]

['1',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'OCCUPATION_TYPE',
 'client_bureau_balance_MONTHS_BALANCE_min_mean',
 'DAYS_BIRTH',
 'bureau_CREDIT_ACTIVE_count_norm',
 'EXT_SOURCE_1^2',
 'EXT_SOURCE_1 EXT_SOURCE_2',
 'EXT_SOURCE_1 EXT_SOURCE_3',
 'EXT_SOURCE_1 OCCUPATION_TYPE',
 'EXT_SOURCE_1 client_bureau_balance_MONTHS_BALANCE_min_mean',
 'EXT_SOURCE_1 DAYS_BIRTH',
 'EXT_SOURCE_1 bureau_CREDIT_ACTIVE_count_norm',
 'EXT_SOURCE_2^2',
 'EXT_SOURCE_2 EXT_SOURCE_3',
 'EXT_SOURCE_2 OCCUPATION_TYPE',
 'EXT_SOURCE_2 client_bureau_balance_MONTHS_BALANCE_min_mean',
 'EXT_SOURCE_2 DAYS_BIRTH',
 'EXT_SOURCE_2 bureau_CREDIT_ACTIVE_count_norm',
 'EXT_SOURCE_3^2',
 'EXT_SOURCE_3 OCCUPATION_TYPE',
 'EXT_SOURCE_3 client_bureau_balance_MONTHS_BALANCE_min_mean',
 'EXT_SOURCE_3 DAYS_BIRTH',
 'EXT_SOURCE_3 bureau_CREDIT_ACTIVE_count_norm',
 'OCCUPATION_TYPE^2',
 'OCCUPATION_TYPE client_bureau_balance_MONTHS_BALANCE_min_mean',
 'OCCUPATION_TYPE DAYS_BIRTH',
 'OCCUPATION_TYPE bureau_CREDIT_ACTIV

In [21]:
# Create a dataframe of the features 
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(cols_ext_data_test))

# Add in the target
poly_features['TARGET'] = poly_target

# Find the correlations with the target
poly_corrs = poly_features.corr()['TARGET'].sort_values()

# Display most negative and most positive
print(poly_corrs.head(10))
print(poly_corrs.tail(10))

EXT_SOURCE_2 EXT_SOURCE_3                                   -0.193939
EXT_SOURCE_2 EXT_SOURCE_3 bureau_CREDIT_ACTIVE_count_norm   -0.190876
EXT_SOURCE_1 EXT_SOURCE_2 EXT_SOURCE_3                      -0.189605
EXT_SOURCE_2^2 EXT_SOURCE_3                                 -0.176428
EXT_SOURCE_2 EXT_SOURCE_3^2                                 -0.172282
EXT_SOURCE_1 EXT_SOURCE_2                                   -0.166625
EXT_SOURCE_1 EXT_SOURCE_3                                   -0.164065
EXT_SOURCE_2 EXT_SOURCE_3 OCCUPATION_TYPE                   -0.162234
EXT_SOURCE_2                                                -0.160295
EXT_SOURCE_1 EXT_SOURCE_3 bureau_CREDIT_ACTIVE_count_norm   -0.159629
Name: TARGET, dtype: float64
EXT_SOURCE_2^2 DAYS_BIRTH                                                  0.149313
EXT_SOURCE_3 DAYS_BIRTH                                                    0.150109
EXT_SOURCE_1 EXT_SOURCE_3 DAYS_BIRTH                                       0.151816
EXT_SOURCE_1 EXT_SO

In [22]:
# Put test features into dataframe
poly_features_test = pd.DataFrame(poly_features_test, 
                                  columns = poly_transformer.get_feature_names(cols_ext_data_test))

# Merge polynomial features into training dataframe
poly_features['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# Merge polnomial features into testing dataframe
poly_features_test['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# Align the dataframes
app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join = 'inner', axis = 1)

# Print out the new shapes
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

Training data with polynomial features shape:  (307511, 422)
Testing data with polynomial features shape:   (48744, 422)


In [23]:
app_train = pd.concat([app_train_poly, poly_target], axis=1)
app_test = app_test_poly

In [24]:
app_train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,client_bureau_balance_MONTHS_BALANCE_min_mean^2 DAYS_BIRTH,client_bureau_balance_MONTHS_BALANCE_min_mean^2 bureau_CREDIT_ACTIVE_count_norm,client_bureau_balance_MONTHS_BALANCE_min_mean DAYS_BIRTH^2,client_bureau_balance_MONTHS_BALANCE_min_mean DAYS_BIRTH bureau_CREDIT_ACTIVE_count_norm,client_bureau_balance_MONTHS_BALANCE_min_mean bureau_CREDIT_ACTIVE_count_norm^2,DAYS_BIRTH^3,DAYS_BIRTH^2 bureau_CREDIT_ACTIVE_count_norm,DAYS_BIRTH bureau_CREDIT_ACTIVE_count_norm^2,bureau_CREDIT_ACTIVE_count_norm^3,TARGET
0,100002,0.083459,0.101419,0.085002,0.079616,0,202500.0,24700.5,351000.0,0.081830,...,-7.550469e+06,60.017843,-2.528672e+09,20100.135093,-0.159774,-8.468590e+11,6.731589e+06,-53.508651,0.000425,True
1,100003,0.083459,0.069993,0.085002,0.083249,0,270000.0,35698.5,1129500.0,0.074946,...,-1.825708e+07,81.897635,-9.275152e+09,41606.480408,-0.186638,-4.712058e+12,2.113735e+07,-94.817941,0.000425,False
2,100004,0.054783,0.101419,0.072437,0.079616,0,67500.0,6750.0,135000.0,0.081830,...,-2.074109e+07,75.479690,-1.197075e+10,43563.217383,-0.158533,-6.908939e+12,2.514258e+07,-91.497291,0.000333,False
3,100006,0.083459,0.069993,0.085002,0.079616,0,135000.0,29686.5,297000.0,0.081830,...,-2.069644e+07,85.353457,-1.191927e+10,49155.831752,-0.202722,-6.864416e+12,2.830929e+07,-116.749316,0.000481,False
4,100007,0.083459,0.101419,0.085002,0.079616,0,121500.0,21865.5,513000.0,0.081830,...,-2.170595e+07,75.479690,-1.311039e+10,45589.732694,-0.158533,-7.918677e+12,2.753620e+07,-95.753649,0.000333,False


# Baseline

For a naive baseline, we could guess the same value for all examples on the testing set.  We are asked to predict the probability of not repaying the loan, so if we are entirely unsure, we would guess 0.5 for all observations on the test set. This  will get us a Reciever Operating Characteristic Area Under the Curve (AUC ROC) of 0.5 in the competition ([random guessing on a classification task will score a 0.5](https://stats.stackexchange.com/questions/266387/can-auc-roc-be-between-0-0-5)).

Since we already know what score we are going to get, we don't really need to make a naive baseline guess. Let's use a slightly more sophisticated model for our actual baseline: Logistic Regression.

## Logistic Regression Implementation

Here I will focus on implementing the model rather than explaining the details, but for those who want to learn more about the theory of machine learning algorithms, I recommend both [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/) and [Hands-On Machine Learning with Scikit-Learn and TensorFlow](http://shop.oreilly.com/product/0636920052289.do). Both of these books present the theory and also the code needed to make the models (in R and Python respectively). They both teach with the mindset that the best way to learn is by doing, and they are very effective! 

To get a baseline, we will use all of the features after encoding the categorical variables. We will preprocess the data by filling in the missing values (imputation) and normalizing the range of the features (feature scaling). The following code performs both of these preprocessing steps.

In [25]:
from sklearn.preprocessing import MinMaxScaler, Imputer
train_labels = app_train['TARGET']
# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()
    
# Feature names
features = list(train.columns)

# Copy of the testing data
test = app_test.copy()

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(app_test)

# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

/home/burin/.env/lib/python3.5/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Training data shape:  (307511, 422)
Testing data shape:  (48744, 422)


In [26]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(train, train_labels)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [27]:
# Make predictions
# Make sure to select the second column only
log_reg_pred = log_reg.predict_proba(test)[:, 1]

In [28]:
# Submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()

,SK_ID_CURR,TARGET
0,100001,0.059900
1,100005,0.136688
2,100013,0.042421
3,100028,0.070366
4,100038,0.123523


In [29]:
# Save the submission to a csv file
submit.to_csv('log_reg_baseline_Part2-V3.csv', index = False)